In [1]:
#! venv/bin/ python3
# *-encoding: utf8 -*

import requests as rqs
from bs4 import BeautifulSoup
import datetime
import time


def unixTimeStampConvert(num):
    convert = datetime.datetime.fromtimestamp(int(num)).strftime('%Y-%m-%d %H:%M:%S')
    return convert

In [2]:
# CONEXION AL SERVIDOR (SITIO WEB), Y REQUEST (PETICION).
# **********************************************************
# A que sitio me quiero conectar, parametro y URI
myurl = "https://votaciones.hcdn.gob.ar"
# Los siguientes datos los saque de la misma pagina, mirando el codigo fuente para ver
# como enviaba las peticiones y cual debia enviar para obtener todos los datos =)
formQuery = {'anoSearch': -1, 'txtSearch': ''}
endPoint = "https://votaciones.hcdn.gob.ar/votaciones/search"

# Peticion al servidor
r = rqs.post(endPoint, data=formQuery)

# Estado de la conexion
status = r.status_code

if status == 200:
    print(f'Conectando a {myurl}')
    print(f'Estado de la conexion :: {status}')
else:
    print('La conexion no se establecio correctamente')
    print(f'Codigo estado : {status}')
    quit()
# **********************************************************

Conectando a https://votaciones.hcdn.gob.ar
Estado de la conexion :: 200


In [3]:
# Antes de programar el scraper deberiamos saber bien cuanl es el objeto html que contiene la informacion que
# queremos por ejemplo, cual es el tag que contiene la info que queremos.
# podemos identificar el objeto con la informacion a traves del nombre del Tag, del selctor css, "class", del id, etc

# El elemento que me interesa tiene la clase "row-acta"
clase = "row-acta"

# Creo el objeto BeautifulSoup
soup = BeautifulSoup(r.content, 'html.parser')

# El tipo de dato es un result set de beautiful soup, es un iterable, algo asi como una lista
datos = soup.find_all("tr", class_=clase)

In [4]:
# EXTRACCION DE LAS URLS DONDE ESTAN LOS DETALLES DE CADA VOTACION
# **********************************************************
urlActas = list()
for t in datos:
    # Columnas
    cols = t.find_all('td')
    idActa = t['id']  # Seria Clave principal
    date = unixTimeStampConvert(t['data-date']) # Tengo que convertir el formato de la fecha en human friendly =)
    # Me voy a guardar los datos que extraje en una lista de tuplas
    urlActas.append((myurl + cols[4].find('a')['href'], idActa, date))
# **********************************************************

In [7]:
# RECORRIDO DE LAS ACTAS Y SCRAPING DE LOS DETALLES - ACTAS Y VOTACION
# **********************************************************
actas = list()
votaciones = list()
for a in urlActas:
    #Conexion a cada Acta # *********************************
    r = rqs.get(a[0])
    soup = BeautifulSoup(r.content, 'html.parser')
    
    # idActa = a[1] este es el id que figura en el sitio, pero voy a usar otro
    dateActa = a[2]
    urlActa = a[0]

    # VAmos a recuperar estos datos -> Período 123 - Reunión 40 - Acta 31
    text = soup.find('h5').text.split('-')
    periodo = text[0].split()[1]
    reunion = text[1].split()[1]
    numeroActa = text[2].split()[1]
    idSite = a[1]
    idActa = idVotacion = '{}-{}-{}'.format(periodo, reunion, numeroActa)

    # Acta # *********************************
    div = soup.find('div', class_='white-box')
    h3 = div.find_all('h3')
    h4 = div.find_all('h4')

    titulo = h4[0].text.strip().lower()
    presidente = h4[1].find('b').text
    resolucion = h3[0].text

    actas.append({
        idActa:{
            'idActa':idActa,
            'periodo': periodo,
            'reunion': reunion,
            'numeroActa': numeroActa,
            'titulo': titulo,
            'presidente': presidente,
            'resolucion': resolucion
        }}
    )

    # Votacion # *********************************
    tabla = soup.find('table', id='myTable')
    rows = tabla.find_all('tr')
    count = 1
    votos = list()
    for r in rows:
        cols = [x for x in r.find_all('td')]
        if cols:
            try:
                idDip = cols[0].find('div')['id'].split('-')[1]
            except TypeError:
                pass
            
            voto = cols[4].text.strip().lower()
            dichos = cols[5].text.strip().lower()
            
            votos.append({
                    'idVoto': count,
                    'voto': voto,
                    'numeroActa': numeroActa,
                    'dip': idDip,
                    'dichos': dichos               
            })
            count += 1
            
    if not votos:
        continue
    else:
        votaciones.append({idVotacion: votos})
        
    time.sleep(1)
# **********************************************************

KeyboardInterrupt: 

In [8]:
actas

[{'137-12-27': {'idActa': '137-12-27',
   'periodo': '137',
   'reunion': '12',
   'numeroActa': '27',
   'titulo': 'expediente 0016-pe-2019 - o.d. 2 - ley de solidaridad social y reactivación productiva en el marco de la emergencia pública. artículos 57 a 59. 20/12/2019 - 10:07',
   'presidente': 'MASSA, Sergio Tomás',
   'resolucion': 'AFIRMATIVO'}},
 {'137-12-26': {'idActa': '137-12-26',
   'periodo': '137',
   'reunion': '12',
   'numeroActa': '26',
   'titulo': 'expediente 0016-pe-2019 - o.d. 2 - ley de solidaridad social y reactivación productiva en el marco de la emergencia pública. artículos 54. 20/12/2019 - 09:56',
   'presidente': 'MASSA, Sergio Tomás',
   'resolucion': 'AFIRMATIVO'}},
 {'137-12-25': {'idActa': '137-12-25',
   'periodo': '137',
   'reunion': '12',
   'numeroActa': '25',
   'titulo': 'expediente 0016-pe-2019 - o.d. 2 - ley de solidaridad social y reactivación productiva en el marco de la emergencia pública. artículos 53. 20/12/2019 - 09:53',
   'presidente': '

In [9]:
votaciones

[{'137-12-27': [{'idVoto': 1,
    'voto': 'ausente',
    'acta': '137-12-27',
    'dip': 'A50768',
    'dichos': ''},
   {'idVoto': 2,
    'voto': 'ausente',
    'acta': '137-12-27',
    'dip': 'A50768',
    'dichos': ''},
   {'idVoto': 3,
    'voto': 'afirmativo',
    'acta': '137-12-27',
    'dip': 'A4759',
    'dichos': ''},
   {'idVoto': 4,
    'voto': 'afirmativo',
    'acta': '137-12-27',
    'dip': 'A4608',
    'dichos': ''},
   {'idVoto': 5,
    'voto': 'negativo',
    'acta': '137-12-27',
    'dip': 'A51069',
    'dichos': ''},
   {'idVoto': 6,
    'voto': 'afirmativo',
    'acta': '137-12-27',
    'dip': 'A51254',
    'dichos': ''},
   {'idVoto': 7,
    'voto': 'afirmativo',
    'acta': '137-12-27',
    'dip': 'A51078',
    'dichos': ''},
   {'idVoto': 8,
    'voto': 'afirmativo',
    'acta': '137-12-27',
    'dip': 'A51042',
    'dichos': ''},
   {'idVoto': 9,
    'voto': 'ausente',
    'acta': '137-12-27',
    'dip': 'A51187',
    'dichos': ''},
   {'idVoto': 10,
    'voto'